In [ ]:
%matplotlib ipympl
# %matplotlib widget
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
# pip install python-maze-generator
from python_maze_generator import multithreaded_maze
import cv2

import random
# set seed
random.seed(0)

In [ ]:

# We will create a random maze of size n x n
def create_maze(n,it,thre, scaleUP=10):
    # create a maze
    maze = multithreaded_maze.generate_mazes(n,n,it,thre)
    # convert PIL.Image.Image image mode=RGB size=234x234 to cv2 image
    maze.draw(solved=False, show=False)
    img = maze.image
    img = np.array(img.convert('1'))
    # scale up the image using np
    img = np.repeat(np.repeat(img, scaleUP, axis=0), scaleUP, axis=1)

    return maze, img

In [ ]:
maze, img = create_maze(10,it = 10, thre = 4, scaleUP=4)

In [ ]:
solution_path =[]
solution_path_scaled = []
scale_shit = img.shape[0]/len(maze.m)

# Draw scaled up path
for i,row in enumerate(maze.m[:]):
    for j,col_item in enumerate(row):
        print(col_item, end=' ')
        if col_item == maze.solved_path:
            solution_path.append([i,j])
            solution_path_scaled.append([i*scale_shit,j*scale_shit])
            continue
        if i == 0 or i == len(maze.m)-1 or j == 0 or j == len(maze.m[0])-1:
            continue
        if col_item == maze.open_wall:
            # if above and below is a solved_path, then add to solution_path
            if maze.m[i-1][j] == maze.solved_path and maze.m[i+1][j] == maze.solved_path:
                solution_path.append([i,j])
                solution_path_scaled.append([i*scale_shit,j*scale_shit])
                continue
            # if left and right is a solved_path, then add to solution_path
            if maze.m[i][j-1] == maze.solved_path and maze.m[i][j+1] == maze.solved_path:
                solution_path.append([i,j])
                solution_path_scaled.append([i*scale_shit,j*scale_shit])
                continue
    print()

In [ ]:
# fig, ax = plt.subplots(1,2,figsize=(10, 5))
# # draw the path

# ax[0].imshow(img, cmap=plt.gray())
# # for i in range(len(solution_path_scaled)):
# #     ax[0].scatter(solution_path_scaled[i][1], solution_path_scaled[i][0], color='r', s=10)

# # move x axis to the top
# ax[0].xaxis.tick_top()
# ax[1].xaxis.tick_top()

# # add a title
# ax[0].set_title('World')
# ax[1].set_title('Agent View')
# plt.show()

In [ ]:
class Agent:
    def __init__(self, img, startPose, ax, lidar_sample=20, lidarRange =20):
        self.org_img = img.copy()
        self.pose_rc = startPose
        
        # lidar range
        self.lidarRange = lidarRange
        self._body_size = self.lidarRange // 5
        # ax to draw self
        self.world_ax = ax[0]
        self.agent_map_ax = ax[1]
        self.lidar_sweep_res = lidar_sample
        self.lidar_step_res = self._body_size//2
        self.lidar_points = []
        # draw self
        self.drawSelf()

    def draw_fesh_map(self, new_img=None):
        # if a new image is provided, then draw the new image
        if new_img is not None:
            self.org_img = new_img.copy()
            self.world_ax.imshow(new_img, cmap=plt.gray())
        else:
            self.world_ax.imshow(self.org_img, cmap=plt.gray())

    def drawSelf(self):
        # draw an orange circle
        self.draw_fesh_map()

        self.drawLidar()
        self.world_ax.add_patch(plt.Circle((self.pose_rc[1], self.pose_rc[0]), self._body_size, color='green'))
        self.agent_map_ax.add_patch(plt.Circle((self.pose_rc[1], self.pose_rc[0]), self._body_size, color='green'))
        

    def drawLidar(self):
        # draw lidar
        self.world_ax.add_patch(plt.Circle((self.pose_rc[1], self.pose_rc[0]), self.lidarRange*10, color='blue', fill=False))
        # draw lidar points
        # sample the map to get lidar points
        # sweep the lidar 360 degrees and 
        for ray in np.arange(0,2*np.pi,2*np.pi/self.lidar_sweep_res):
            # ray trace to the 1st intersection
            for r in np.arange(0,self.lidarRange, self.lidar_step_res):
                # get the point
                x = self.pose_rc[1] + r*np.cos(ray)
                y = self.pose_rc[0] + r*np.sin(ray)
                # check if the point is in the map
                if x < 0 or x > img.shape[1] or y < 0 or y > img.shape[0]:
                    break
                if img[int(y),int(x)] == 0:
                    # draw the point
                    self.lidar_points.append([y,x])
                    # draw on agent map
                    self.world_ax.scatter(x, y, color='r', s=10)
                    self.agent_map_ax.scatter(x, y, color='r', s=10)

                    break


In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10, 5))
ax[0].xaxis.tick_top()
ax[1].xaxis.tick_top()

# add a title
ax[0].set_title('World')
ax[1].set_title('Agent View')

bot1 = Agent(img, solution_path_scaled[0], ax, lidar_sample=200, lidarRange = 2*scale_shit)

In [ ]:
# import numpy as np
# import scipy.stats as st
# # %matplotlib notebook
# import matplotlib.pyplot as plt
# import matplotlib.animation as animation
# # from IPython.display import HTML
# # from matplotlib.animation import PillowWriter

# X = np.linspace(-5,5, 100)
# number_of_frames = np.size(X)
# N_max = 100
# N = np.arange(1,N_max+1)
# h = 1/np.sqrt(N)

# def update_plot(n):
#     #plt.cla()
#     lines.set_data(X[:n], h[:n])
#     lines2.set_data(X[:n], h[:n]*-1)
#     ax.set_title("n = {}".format(n))
#     return lines, lines2

# fig = plt.figure()
# ax = plt.axes(xlim=(-5, 5), ylim=(-1, 1))
# lines, = ax.plot([], [], 'y-', lw=2, label="densité")
# lines2, = ax.plot([], [], 'b--', lw=3, label="densité2")

# ani = animation.FuncAnimation(fig, update_plot, frames=number_of_frames, repeat=False )
# plt.show()
# # ani.save('lines_ani2.gif', writer='pillow')
